In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/PSR

/content/drive/MyDrive/PSR


In [ ]:
!pip install keras-flops

In [ ]:
import os
import logging
import tensorflow as tf
from keras_flops import get_flops
from keras.utils.layer_utils import count_params
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import inception_resnet_v2
from tensorflow.keras.applications import efficientnet 
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import nasnet
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.applications import xception

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("LIBRARIES LOADED")

LIBRARIES LOADED


In [ ]:
#SET COMMON HPs
BATCH_SIZE = 4
NUM_CLASSES = 199

#SET DATA PATHS
MAIN_DATA_DIR = "ds/"
TRAIN_DATA_DIR = MAIN_DATA_DIR + "train/"
TEST_DATA_DIR = MAIN_DATA_DIR + "test/"
VALIDATION_DATA_DIR = MAIN_DATA_DIR + "val/"

#Load model Function
def load_m(directory, model_name, custom_objects=None):
    if not os.path.exists(directory):
        print("Model File Does Not Exist!!")
        return 
    model = load_model(directory + "/" + model_name + ".h5", custom_objects=custom_objects)
    return model

def cost_compute(directory, model_name, model):
    params = float("{0:.2f}".format(model.count_params() / 10 ** 6))
    flops  = float("{0:.2f}".format(get_flops(Model(model.input, model.output), batch_size=1)/ 10 ** 9))
    model_file_size = float("{0:.2f}".format(os.stat(directory + '/' + model_name + '.h5').st_size / 10 ** 6))
    print("#COST FOR", MODEL_NAME)
    print("FLOPS:", flops, "G")
    print("PARAMETER SIZE:", params, "M")
    print("MODEL FILE SIZE:", model_file_size, "MB")

#DATA GENERATORS
def create_data_generator(input_shape, pre_process=None):
    print("INPUT SIZE -->", input_shape, "\n")
    nb_test_samples = 0
    nb_val_samples = 0
    num_classes = 0
    test_generator = None
    validation_generator = None

    test_datagen = ImageDataGenerator(preprocessing_function=pre_process)
    val_datagen = ImageDataGenerator(preprocessing_function=pre_process)

    if not os.path.exists(VALIDATION_DATA_DIR):
        print("VALIDATION DATA DOES NOT EXITS!")
    else:
        print("LOAD VALIDATION SAMPLES...")
        validation_generator = val_datagen.flow_from_directory(
                VALIDATION_DATA_DIR,
                target_size=input_shape,
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                seed=42,
                shuffle=False)

        #CHECK  THE NUMBER OF SAMPLES
        nb_validation_samples = len(validation_generator.filenames)
        if nb_validation_samples == 0:
            print("NO DATA VALIDATION FOUND IN VALIDATION FOLDER!")

    #LOAD TEST DATA
    if not os.path.exists(TEST_DATA_DIR):
        print("TEST DATA DOES NOT EXITS!")
    else:
        print("LOAD TEST SAMPLES...")
        test_generator = test_datagen.flow_from_directory(
                    TEST_DATA_DIR,
                    target_size=input_shape,
                    batch_size=BATCH_SIZE,
                    class_mode='categorical',
                    seed=42,
                    shuffle=False)

        #CHECK  THE NUMBER OF SAMPLES
        nb_test_samples = len(test_generator.filenames)
        if nb_test_samples == 0:
            print("NO DATA TEST FOUND IN TEST FOLDER!") 

    if nb_test_samples > 0 and nb_validation_samples > 0:
        num_classes= len(test_generator.class_indices)
        print("GENERATER ARE SET!")
        print('CLASSES TO TRAIN', num_classes, 'classes')
    
    return (test_generator, nb_test_samples), (validation_generator, nb_validation_samples)

print("BATCH SIZE SET ->", BATCH_SIZE)
print("DATA PATHS SET")
print("ALL CUSTOM FUNCTIONS DEFIEND")

BATCH SIZE SET -> 4
DATA PATHS SET
ALL CUSTOM FUNCTIONS DEFIEND


**DenseNet121 & DenseNet201** 

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "DenseNet121"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-DenseNet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                           

In [ ]:
from tensorflow.keras.layers import Input 

input = Input(shape=(255,255,3))
new_model = tf.keras.models.clone_model(
    model, input_tensors=input, clone_function=None
)

new_model.summary()

Model: "benchmark-onelayeroff-0_0001-DenseNet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 261, 261, 3)  0          ['input_3[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 128, 128, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                           

In [ ]:
new_model.input.shape[1]

255

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.densenet import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = new_model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = new_model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR DenseNet121
  5/299 [..............................] - ETA: 10:42

KeyboardInterrupt: ignored

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.densenet import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR DenseNet121
199/199 [==============================] - 424s 2s/step

#ACCURACY FOR DenseNet121
VAL: 0.9028475711892797
TEST: 0.9120603015075377

#MSE FOR DenseNet121
VAL: 646.643216080402
TEST: 574.4497487437186

#MSLE FOR DenseNet121
VAL: 0.20803797322425166
TEST: 0.10580714496731092

#COST FOR DenseNet121
FLOPS: 5.7 G
PARAMETER SIZE: 7.24 M
MODEL FILE SIZE: 88.36 MB


In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "DenseNet201"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-DenseNet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                           

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.densenet import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL: ", val_accuracy)
print("TEST: ", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR DenseNet201
199/199 [==============================] - 133s 666ms/step

#ACCURACY FOR DenseNet201
VAL:  0.9170854271356784
TEST:  0.9057788944723618

#MSE FOR DenseNet201
VAL: 569.5092127303183
TEST: 520.7311557788945

#MSLE FOR DenseNet201
VAL: 0.15383535314335844
TEST: 0.11376634468530417

#COST FOR DenseNet201
FLOPS: 8.63 G
PARAMETER SIZE: 18.7 M
MODEL FILE SIZE: 226.09 MB


**EfficientNetB0 & EfficientNetB7**

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "EfficientNetB0"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-EfficientNetB0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0  

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.efficientnet import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR EfficientNetB0
199/199 [==============================] - 44s 220ms/step

#ACCURACY FOR EfficientNetB0
VAL: 0.6063651591289783
TEST: 0.7412060301507538

#MSE FOR EfficientNetB0
VAL: 2652.64405360134
TEST: 1288.4434673366834

#MSLE FOR EfficientNetB0
VAL: 0.6815934319893229
TEST: 0.32551498567305137

#COST FOR EfficientNetB0
FLOPS: 0.79 G
PARAMETER SIZE: 4.3 M
MODEL FILE SIZE: 52.5 MB


In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "EfficientNetB7"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-EfficientNetB7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0  

In [ ]:
import pickle

def load_h(directory, history_name):
    if not os.path.exists(directory):
        print("History File Does Not Exist!!")
        return 
    with open(directory + '/' + history_name + '.history', 'rb') as file:
        his = pickle.load(file)
    return his

his = load_h(MODEL_PATH, MODEL_NAME)
his['val_accuracy']

[0.38842281699180603,
 0.7399328947067261,
 0.8364093899726868,
 0.8674496412277222,
 0.8817114233970642,
 0.880033552646637,
 0.8984899520874023,
 0.8783556818962097,
 0.8758389353752136,
 0.8833892345428467,
 0.9026845693588257,
 0.8875839114189148,
 0.899328887462616,
 0.9018456339836121,
 0.9052013158798218,
 0.911912739276886,
 0.8968120813369751,
 0.9102349281311035,
 0.9026845693588257,
 0.9026845693588257,
 0.9026845693588257,
 0.9043624401092529,
 0.9052013158798218,
 0.9043624401092529,
 0.9018456339836121,
 0.9043624401092529,
 0.9001677632331848,
 0.9043624401092529,
 0.9035235047340393,
 0.9052013158798218]

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.efficientnet import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR EfficientNetB7
199/199 [==============================] - 297s 1s/step

#ACCURACY FOR EfficientNetB7
VAL: 0.8299832495812395
TEST: 0.8932160804020101

#MSE FOR EfficientNetB7
VAL: 1165.3827470686767
TEST: 658.3115577889447

#MSLE FOR EfficientNetB7
VAL: 0.2958619073951667
TEST: 0.15625237241378173

#COST FOR EfficientNetB7
FLOPS: 10.49 G
PARAMETER SIZE: 64.61 M
MODEL FILE SIZE: 776.71 MB


**ResNet50V2 & ResNet151V2**

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "ResNet50V2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-ResNet50V2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                            

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.resnet_v2 import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR ResNet50V2
199/199 [==============================] - 94s 472ms/step

#ACCURACY FOR ResNet50V2
VAL: 0.8509212730318257
TEST: 0.8542713567839196

#MSE FOR ResNet50V2
VAL: 953.4388609715243
TEST: 756.3103015075377

#MSLE FOR ResNet50V2
VAL: 0.2680031355448559
TEST: 0.14520722975666547

#COST FOR ResNet50V2
FLOPS: 6.99 G
PARAMETER SIZE: 23.97 M
MODEL FILE SIZE: 288.25 MB


In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "ResNet152V2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-ResNet152V2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                           

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.resnet_v2 import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR ResNet152V2
199/199 [==============================] - 247s 1s/step

#ACCURACY FOR ResNet152V2
VAL: 0.8433835845896147
TEST: 0.8756281407035176

#MSE FOR ResNet152V2
VAL: 1028.5695142378559
TEST: 751.6143216080402

#MSLE FOR ResNet152V2
VAL: 0.3092316912223524
TEST: 0.19486183291461823

#COST FOR ResNet152V2
FLOPS: 21.88 G
PARAMETER SIZE: 58.74 M
MODEL FILE SIZE: 706.53 MB


**NasNetMobile & NasNetMobile**

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "NASNetMobile"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-NASNetMobile"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                         

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.nasnet import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR NASNetMobile
199/199 [==============================] - 59s 294ms/step

#ACCURACY FOR NASNetMobile
VAL: 0.8492462311557789
TEST: 0.8329145728643216

#MSE FOR NASNetMobile
VAL: 982.5067001675042
TEST: 1073.9183417085428

#MSLE FOR NASNetMobile
VAL: 0.20266536639925628
TEST: 0.25427668572704426

#COST FOR NASNetMobile
FLOPS: 1.15 G
PARAMETER SIZE: 4.48 M
MODEL FILE SIZE: 57.36 MB


In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "NASNetLarge"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-NASNetLarge"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 331, 331, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 165, 165, 96  2592        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 165, 165, 96  384         ['stem_conv1[0][0]']             
                                )                          

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.nasnet import preprocess_input
INPUT_SHAPE = (331, 331)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (331, 331) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR NASNetLarge
199/199 [==============================] - 624s 3s/step

#ACCURACY FOR NASNetLarge
VAL: 0.873534338358459
TEST: 0.8907035175879398

#MSE FOR NASNetLarge
VAL: 672.3735343383585
TEST: 741.0

#MSLE FOR NASNetLarge
VAL: 0.1545412052591332
TEST: 0.19856544688484756

#COST FOR NASNetLarge
FLOPS: 47.8 G
PARAMETER SIZE: 85.72 M
MODEL FILE SIZE: 1032.37 MB


**InceptionV3 & InceptionResNetV2**

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "InceptionV3"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-InceptionV3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                          

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.inception_v3 import preprocess_input
INPUT_SHAPE = (299, 299)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)
#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (299, 299) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR InceptionV3
199/199 [==============================] - 140s 705ms/step

#ACCURACY FOR InceptionV3
VAL: 0.8785594639865997
TEST: 0.8907035175879398

#MSE FOR InceptionV3
VAL: 806.1298157453937
TEST: 594.7060301507538

#MSLE FOR InceptionV3
VAL: 0.1832907784136712
TEST: 0.11303514772284161

#COST FOR InceptionV3
FLOPS: 11.47 G
PARAMETER SIZE: 22.21 M
MODEL FILE SIZE: 267.68 MB


In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "InceptionResNetV2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

/usr/local/lib/python3.7/dist-packages/keras/layers/core/lambda_layer.py:299: UserWarning: tensorflow.python.keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  'function_type')


Model: "benchmark-onelayeroff-0_0001-InceptionResNetV2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                    

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
INPUT_SHAPE = (299, 299)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (299, 299) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR InceptionResNetV2
199/199 [==============================] - 292s 1s/step

#ACCURACY FOR InceptionResNetV2
VAL: 0.8802345058626466
TEST: 0.8969849246231156

#MSE FOR InceptionResNetV2
VAL: 841.1700167504188
TEST: 505.2324120603015

#MSLE FOR InceptionResNetV2
VAL: 0.22581164295335002
TEST: 0.10249402405137305

#COST FOR InceptionResNetV2
FLOPS: 26.38 G
PARAMETER SIZE: 54.64 M
MODEL FILE SIZE: 658.69 MB


**Xception**

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "Xception"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-Xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                             

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.xception import preprocess_input
INPUT_SHAPE = (299, 299)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (299, 299) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR Xception
199/199 [==============================] - 206s 1s/step

#ACCURACY FOR Xception
VAL: 0.8576214405360134
TEST: 0.8391959798994975

#MSE FOR Xception
VAL: 836.6675041876047
TEST: 962.7060301507538

#MSLE FOR Xception
VAL: 0.2495238820001263
TEST: 0.19790609219265162

#COST FOR Xception
FLOPS: 16.77 G
PARAMETER SIZE: 21.27 M
MODEL FILE SIZE: 255.57 MB


**MobileNetV2**

In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "MobileNetV2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "benchmark-onelayeroff-0_0001-MobileNetV2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                          

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR MobileNetV2
199/199 [==============================] - 36s 179ms/step

#ACCURACY FOR MobileNetV2
VAL: 0.8567839195979899
TEST: 0.8869346733668342

#MSE FOR MobileNetV2
VAL: 647.8835845896148
TEST: 496.20100502512565

#MSLE FOR MobileNetV2
VAL: 0.18690264230544076
TEST: 0.11432524641423854

#COST FOR MobileNetV2
FLOPS: 0.61 G
PARAMETER SIZE: 2.51 M
MODEL FILE SIZE: 30.75 MB


**ComMonBase**

In [ ]:
#Tweakable parameters
MODEL_KIND = "proposed_model"
MODEL_NAME = "MiniMobileNetV2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "ablation_final-MiniMobileNetV2-39"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                 

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR MiniMobileNetV2
199/199 [==============================] - 34s 171ms/step

#ACCURACY FOR MiniMobileNetV2
VAL: 0.8400335008375209
TEST: 0.8467336683417085

#MSE FOR MiniMobileNetV2
VAL: 1002.356783919598
TEST: 889.0929648241206

#MSLE FOR MiniMobileNetV2
VAL: 0.2631270299634975
TEST: 0.1660191166274771

#COST FOR MiniMobileNetV2
FLOPS: 0.43 G
PARAMETER SIZE: 0.58 M
MODEL FILE SIZE: 7.47 MB


**ComMonBase-KD**

In [ ]:
#Tweakable parameters
MODEL_KIND = "proposed_model"
MODEL_NAME = "MiniMobileNetV2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME + '-KD'

#Load Model
model = load_m(MODEL_PATH, MODEL_NAME)
model.summary()

Model: "ablation_final-PROPOSEDMiniMobileNetV2-39"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                         

In [ ]:
#DATA GENERATORS
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=preprocess_input)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR MiniMobileNetV2
199/199 [==============================] - 34s 171ms/step

#ACCURACY FOR MiniMobileNetV2
VAL: 0.9078726968174204
TEST: 0.9095477386934674

#MSE FOR MiniMobileNetV2
VAL: 638.8283082077052
TEST: 385.46608040201005

#MSLE FOR MiniMobileNetV2
VAL: 0.16114358963673422
TEST: 0.05940666915386783

#COST FOR MiniMobileNetV2
FLOPS: 0.43 G
PARAMETER SIZE: 0.58 M
MODEL FILE SIZE: 2.67 MB


**Ensemble Model**

In [ ]:
class EN_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(EN_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = efficientnet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(EN_PreprocessLayer, self).get_config()
        return config

class Dense_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(Dense_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = densenet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(Dense_PreprocessLayer, self).get_config()
        return config

class INV3_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(INV3_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = inception_v3.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(INV3_PreprocessLayer, self).get_config()
        return config
    
class NN_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(NN_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = nasnet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(NN_PreprocessLayer, self).get_config()
        return config

custom_layers = {'EN_PreprocessLayer':EN_PreprocessLayer, 
                     'Dense_PreprocessLayer':Dense_PreprocessLayer,
                     'NN_PreprocessLayer':NN_PreprocessLayer,
                     'INV3_PreprocessLayer':INV3_PreprocessLayer,
                     }

print("ALL REQUERED PREPROCESSING LAYERS DEFINED")

ALL REQUERED PREPROCESSING LAYERS DEFINED


In [ ]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "EnsembleModel"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND + '/' + MODEL_NAME

#PLOT THE MODEL STRUCTURE
def get_teacher():
    model = load_m(MODEL_PATH, MODEL_NAME, custom_objects=custom_layers)
    if model != None:
        print("ENSEMBLE MODEL LOADED SUCCESSFULLY!")

    return model

model = get_teacher()

ENSEMBLE MODEL LOADED SUCCESSFULLY!


In [ ]:
#DATA GENERATORS
INPUT_SHAPE = (224, 224)

(test_generator, nb_test_samples), (validation_generator, nb_validation_samples) = create_data_generator(INPUT_SHAPE, 
                                                                                                         pre_process=None)

#GET TRUE LABELS
target_val = validation_generator.classes 
target_test = test_generator.classes

print()
#GET PREDICTIONS
print("GET PREDICTIONS FOR", MODEL_NAME)
pred_val = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)
pred_test = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

print()
#EVALUATE ACCURACY
val_accuracy = accuracy_score(target_val, pred_val.argmax(axis=-1))
test_accuracy = accuracy_score(target_test, pred_test.argmax(axis=-1))
print("#ACCURACY FOR", MODEL_NAME)
print("VAL:", val_accuracy)
print("TEST:", test_accuracy)
print()

#EVALUATE MSE
val_mse = mean_squared_error(target_val, pred_val.argmax(axis=-1))
test_mse = mean_squared_error(target_test, pred_test.argmax(axis=-1))
print("#MSE FOR", MODEL_NAME)
print("VAL:", val_mse)
print("TEST:", test_mse)
print()

#EVALUATE MSLE
val_msle = mean_squared_log_error(target_val, pred_val.argmax(axis=-1))
test_msle = mean_squared_log_error(target_test, pred_test.argmax(axis=-1))
print("#MSLE FOR", MODEL_NAME)
print("VAL:", val_msle)
print("TEST:", test_msle)
print()

#EVALUATE COST
cost_compute(MODEL_PATH, MODEL_NAME, model)

INPUT SIZE --> (224, 224) 

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.
LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.
GENERATER ARE SET!
CLASSES TO TRAIN 199 classes

GET PREDICTIONS FOR EnsembleModel
199/199 [==============================] - 1105s 6s/step

#ACCURACY FOR EnsembleModel
VAL: 0.9363484087102177
TEST: 0.9246231155778895

#MSE FOR EnsembleModel
VAL: 403.92797319933
TEST: 347.34170854271355

#MSLE FOR EnsembleModel
VAL: 0.13287039073164109
TEST: 0.06677130381773692

#COST FOR EnsembleModel
FLOPS: 51.18 G
PARAMETER SIZE: 198.48 M
MODEL FILE SIZE: 799.26 MB
